<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3: Adding Interactivity to Panel</h2></div>

<div class="alert alert-warning" role="alert"> <strong>WORK IN PROGRESS:</strong> We are in the progress of updating these materials in anticipation of a tutorial at the 2019 SciPy conference. Work will be complete by the morning of July 8th 2019. Check out <a href="https://github.com/pyviz/holoviz/tree/v0.1.1">this tag</a> to access the materials as they were before these changes started. For the latest version of the tutorial, visit <a href="https://holoviz.org/tutorial">holoviz.org</a>.
</div>


In [ ]:
import panel as pn

pn.extension()

In the previous section we learned the very basics of working with Panel. Specifically we looked at the different types of components, how to update them and how to serve a Panel application or dashboard. However to start building actual apps with Panel we need to be able to add interactivity by linking different components together. In this section we will learn how to link widgets to outputs to start building some simple interactive applications.

## Widgets

One of the most common patterns when building an application is linking a widget to some output and updating it in response. As we briefly mentioned in the last section, a widget is an input control which allows a user to change a ``value`` using some graphical UI. A simple example is a ColorPicker:

In [ ]:
color_picker = pn.widgets.ColorPicker(value='#ff0000')

color_picker

When we change the color the ``value`` updates:

In [ ]:
color_picker.value

As a very simple example we can link the slider ``value`` to the ``background`` parameter on a ``HTML`` pane:

In [ ]:
html = pn.pane.HTML('', width=200, height=200, background=color_picker.value)

color_picker.link(html, value='background')

html

Whenever the color is changed the background of the HTML pane will update. Linking may also be done directly in Javascript, which allows building apps which do not require a live server or notebook kernel. For more detail see the [Links user guide](http://panel.pyviz.org/user_guide/Links.html).

#### Exercise

Use tab-completion on the ``pn.widget`` namespace to discover some other widget, construct a string pane and then link the widget value to the pane ``object`` parameter. Finally display both the widget and the pane in a panel.

## Callbacks

The ability to link parameters is somewhat limited, the more general approach is to write callbacks in response to changes in some parameter, e.g. the ``value`` of a widget. All parameters can be watched using the ``.param.watch`` API, which will call the provided callback with an event object containing the old and new value of the widget.

In this example we will use the earthquake data we started playing with in the previous exercise, so will start by loading it:

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet('../data/earthquakes.parq', columns=['time', 'place', 'mag']).reset_index(drop=True).persist()

Now that it is loaded we will create a slider which we will eventually use to select the row of the dataframe that we want to display:

In [ ]:
row_slider = pn.widgets.IntSlider(value=0, start=0, end=len(df)-1)

Next we create a Pane to display the current row of the dataframe:

In [ ]:
row_pane = pn.panel(df.loc[row_slider.value].compute())

Now that we have defined both the widget and the object we want to update we can declare a callback to link the two. As we learned in the previous section assigning a new value to the ``object`` of a pane will update the display. In the callback we select the row of the dataframe and then assign it to the ``pane.object``.

In [ ]:
def df_callback(event):
    row_pane.object = df.loc[event.new].compute()

Lastly we actually have to register this callback, to do so we provide the callback and the parameter we want to trigger the event on the slider's ``.param.watch`` method:

In [ ]:
row_slider.param.watch(df_callback, 'value')

Now that everything is connected up we can put both the widget and the pane in a panel and display them:

In [ ]:
pn.Row(row_slider, row_pane)

## Declaring interactive components

There are a variety of ways of linking different components in Panel, and while writing callbacks like above is the most flexible it is also quite verbose and can quickly become quite messy. A different approach is to express dependencies between the parameters of one object and declare those as inputs to a function.

As a very simple example we will declare a ``TextInput`` widget and then write a function, which uses Markdown syntax to convert the text into a title. Using the ``pn.depends`` decorator we can then declare that this function depends on the ``value`` of the widget. Finally we lay out the widget and the function:

In [ ]:
text_input = pn.widgets.TextInput(value='A title')

@pn.depends(text_input.param.value)
def title_text(value):
    return '## ' + value

pn.Row(text_input, title_text)

When entering some text into the widget (and pressing enter) we will see that it appears as a title. In this way we can easily declare dynamic components which depend on the value of a widget (or any other component which has parameters).

#### Exercise

Declare two ``Spinner`` widgets with an initial value of 1, then declare a function which depends on the values of both widgets and adds them together. Finally lay out the two widgets and the function in a Panel:

<details><summary>Solution</summary><br>

```python
a = pn.widgets.Spinner(value=1, width=60)
b = pn.widgets.Spinner(value=1, width=60)

@pn.depends(a.param.value, b.param.value)
def adder(a, b):
    return a + b

pn.Row(a, '+', b, '=', adder)
```
    
</details>

Now that we have learned not only how to display different types of data, lay it out but also how to link parameters and build interactive components we can start building actual apps and dashboards. Before we move on to plotting and visualization let us quickly recap what we have learned by adding interactivity to [the dashboard we previously build](./exercises/Building_a_Dashboard.ipynb).